In [ ]:
import boto3
import nbformat
from nbconvert import HTMLExporter
import datetime
import os 
from jinja2 import Environment, FileSystemLoader

# Initialize client with manual credentials
s3_client = boto3.client(
    's3',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    region_name=AWS_REGION,
    endpoint_url=AWS_ENDPOINT_URL_S3
)

html_exporter = HTMLExporter(template_name='classic')

# Rest of your script...
paginator = s3_client.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=BUCKET_NAME)

for page in pages:
    for obj in page.get('Contents', []):
        key = obj['Key']
        if key.endswith('.ipynb'):
            print(f"Processing: {key}")
            response = s3_client.get_object(Bucket=BUCKET_NAME, Key=key)
            notebook_content = response['Body'].read().decode('utf-8')
            # Parse and convert to HTML
            notebook_node = nbformat.reads(notebook_content, as_version=4)
            (body, _) = html_exporter.from_notebook_node(notebook_node)
            
            # Save locally (replacing .ipynb extension with .html)
            output_filename = key.split('/')[-1].replace('.ipynb', '.html')
            with open(output_filename, 'w', encoding='utf-8') as f:
                f.write(body)